# Performance_EnergyLandscape.ipynb

### Imports and Setup



In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Import the functions we created
from Functions import (
    run_experiment,    # for generating data 
    load_and_merge_data,
    calculate_performance, 
    calculate_fail_rate,
    plot_performance_and_rt,
    compute_non_regret_performance,
    plot_non_regret,
    full_analysis_pipeline,
    plot_energy_landscapes
)

### Define the parameters first

In [ ]:
params = {
    'n_E': 60,
    'n_I': 50,
    'n_NSE': 280,
    'gamma_ee': 1.2,
    'gamma_ei': 0.0,
    'gamma_ie': 0.0,
    'gamma_ii': 0.0,
    'top_down': False,
    'n_Ctr': 500,
    'S': 0.3,
    'R': 1.0
}
coherence_index = 0  # [0, 3.2, 6.4, 12.8, 25.6, 51.2]


In [ ]:
# Define a coherence list and pick an index
coherence_list = [0, 3.2, 6.4, 12.8, 25.6, 51.2]

# Generate data by running multiple trials
Result_list, total_EL_RT, total_ER_RT, NO_decision, EL_firing, ER_firing = run_experiment(
    params=params,
    coherence_list=coherence_list,
    coherence_index=coherence_index, # pick 3.2
    num_trial=10,
    threshold=60,
    trial_len=2000,
    BG_len=300,
    input_amp=0.00156,
    trial_n='0'
)

# Summarize
print("Number of Trials:", len(Result_list))
print("E_L Firing Trials:", np.sum(EL_firing))
print("E_R Firing Trials:", np.sum(ER_firing))
print("No Decision Trials:", np.sum(NO_decision))

# If you look in your working directory, you should see a file:
# "EE1.2_EI0.0_IE0.0_II0.0_0_trialCount10_trialN0.pkl"


## Example: Load and Merge Data from Multiple Files


In [ ]:
import os

# Suppose you have multiple .pkl files for different coherence or repeated runs.
# For example, let's imagine you generated them with different 'trial_n' or num_trial.
# We'll just show how you'd merge them if you had a list of file paths.

# Example list of file paths (replace with real files you have)
file_paths_example = [
    "EE1.2_EI0.0_IE0.0_II0.0_0_trialCount10_trialN0.pkl",
    # Add more if you've done multiple runs...
]

# Merge them (shifting keys by step=100 for each file)
merged_data, final_offset = load_and_merge_data(file_paths_example, initial_offset=0, step=100)
print("Merged Data size:", len(merged_data))


FileNotFoundError: [Errno 2] No such file or directory: 'GammaEE0.2_GammaEI0.0_GammaIE0.0_GammaII0.0_coh3.2_trialCount10_trialNTest.pkl'

## 3. Compute Performance / RT / Fail Rate

In [ ]:
# Let's compute performance for the merged_data example:
performance, avg_er_rt, std_er_rt = calculate_performance(merged_data)
fail_rate = calculate_fail_rate(merged_data)

print("Performance (fraction of E_R decisions):", performance)
print("Avg E_R Reaction Time:", avg_er_rt)
print("Std E_R Reaction Time:", std_er_rt)
print("Fail Rate:", fail_rate)


## 4. Plot a Psychometric Curve for Multiple Conditions


In [ ]:
# If you want to compare multiple parameter sets (e.g., Pol0 vs. Pol2), 
# you'd produce or load data for each set of gamma or 'pol' parameter, 
# store the performance in a dict keyed by e.g. "coh0", "coh3.2", etc.
# Then call 'plot_performance_and_rt'.

# For demonstration, let's assume we have two dicts: results_pol0, results_pol2
# Each is keyed like: "coh0" -> {"Performance": p, "Avg_ER_Reaction_Time": r, "Std_ER_Reaction_Time": s}, etc.

# We'll do a quick fake example:

coherence_list_for_plot = [0, 3.2, 6.4, 12.8, 25.6, 51.2]

results_pol0 = {
    "coh0":   {"Performance": 0.5, "Avg_ER_Reaction_Time": 350, "Std_ER_Reaction_Time": 40},
    "coh3.2": {"Performance": 0.6, "Avg_ER_Reaction_Time": 330, "Std_ER_Reaction_Time": 35},
    "coh6.4": {"Performance": 0.7, "Avg_ER_Reaction_Time": 320, "Std_ER_Reaction_Time": 30},
    "coh12.8":{"Performance": 0.75,"Avg_ER_Reaction_Time": 310, "Std_ER_Reaction_Time": 25},
    "coh25.6":{"Performance": 0.8, "Avg_ER_Reaction_Time": 300, "Std_ER_Reaction_Time": 20},
    "coh51.2":{"Performance": 0.9, "Avg_ER_Reaction_Time": 290, "Std_ER_Reaction_Time": 15},
}

results_pol2 = {
    "coh0":   {"Performance": 0.52, "Avg_ER_Reaction_Time": 360, "Std_ER_Reaction_Time": 42},
    "coh3.2": {"Performance": 0.63, "Avg_ER_Reaction_Time": 345, "Std_ER_Reaction_Time": 38},
    "coh6.4": {"Performance": 0.71, "Avg_ER_Reaction_Time": 325, "Std_ER_Reaction_Time": 32},
    "coh12.8":{"Performance": 0.77,"Avg_ER_Reaction_Time": 315, "Std_ER_Reaction_Time": 28},
    "coh25.6":{"Performance": 0.82, "Avg_ER_Reaction_Time": 305, "Std_ER_Reaction_Time": 22},
    "coh51.2":{"Performance": 0.88, "Avg_ER_Reaction_Time": 295, "Std_ER_Reaction_Time": 18},
}

plot_performance_and_rt(
    coherence_list_for_plot,
    results_polA=results_pol0,
    results_polB=results_pol2,
    labelA="Pol0",
    labelB="Pol2"
)


## 5. Compute and Plot Non-Regret


In [ ]:
# We can compute non-regret from a given 'Result_list' or merged dict
bm = np.arange(0, 12, 0.6)  # example benchmark array

nr_data_example = compute_non_regret_performance(merged_data, bm)
print("Non-Regret data shape:", len(nr_data_example))

# Suppose you have multiple 'pol' conditions; you'd store them in a dictionary 
# and then plot with 'plot_non_regret'.
NR_results_dict = {
    "Pol0": nr_data_example,   # this is a single example
    # "Pol2": other array,
    # ...
}
labels_dict = {"Pol0": "Pol0, GammaEE=0.2"} # or whatever label you want

plot_non_regret(bm, NR_results_dict, labels_dict, title="Non-Regret Example")


## 6. Using the Full Analysis Pipeline


In [ ]:
# If you have a dictionary of file paths grouped by coherence values,
# you can use 'full_analysis_pipeline' to load everything, compute performance, 
# and do non-regret automatically.

file_paths_dict = {
    0:   ["GammaEE0.2_GammaEI0.0_GammaIE0.0_GammaII0.0_coh0_trialCount10_trialNTest.pkl"],
    3.2: ["GammaEE0.2_GammaEI0.0_GammaIE0.0_GammaII0.0_coh3.2_trialCount10_trialNTest.pkl"],
    # etc...
}
benchmark_array = np.arange(0, 12, 0.6)

all_merged_results, perf_dict, non_regret_dict = full_analysis_pipeline(file_paths_dict, benchmark_array)

print("Performance dict:\n", perf_dict)
print("Non-Regret dict:\n", non_regret_dict)


## 7. Plot Energy Landscapes


In [ ]:
# Finally, we can plot the energy landscape for a particular 'Result_list' 
# (or merged_data) using 'plot_energy_landscapes'.

# 'time_window' is a list of (start_ms, end_ms) relative to reaction start time.
time_windows = [
    (0, 50),     # from 0 ms to 50 ms after reaction start
    (50, 100),   # from 50 ms to 100 ms
]

# For demonstration, let's assume we want excitatory difference (EXC=True) 
# and a limit of 40. We'll also plot full scale.

plot_energy_landscapes(
    Result_list,       # or merged_data
    time_window=time_windows,
    plot_full_scale=True,
    EXC=True,
    lim=40
)
